# Ollama Inference Notebook

This notebook contains the scripts for LLM inference using Ollama containing running on the docker host. This is the playground for testing purpose.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [2]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import requests
# ...
# global constants
ollama_url = "http://ollama:11434"
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.22.1
pandas version: 2.0.3


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

In [4]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [5]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("llm_rag_ollama_text_processing")
print(df.describe())
print(param)

                                                     text
count                                                  19
unique                                                 19
top     software at incredibly low prices ( 86 % lower...
freq                                                    1
{'options': {'params': {'mode': 'stage', 'algo': 'ollama_phishing_detection', 'model': '"llama3"', 'prompt': '"You will examine if the email content given by the user is phishing. Only output **Phishing** if the content is phishing. Only output **Legit** if the email is legitimate. Do not give extra information."'}, 'args': ['text'], 'feature_variables': ['text'], 'model_name': 'ollama_phishing_detection', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'disabled': False, 'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '60000', 'max_inputs': '100000', 'max_memory_usage_mb': '40

## Stage 2 - create and initialize a model

In [ ]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    model['hyperparameter'] = 42.0
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(init(df,param))

## Stage 3 - fit the model

In [ ]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

In [5]:
import requests
import json
def oneshot(prompt, llm):
    uri = "http://ollama:11434/api/chat"
    headers = {'Content-Type': 'application/json'}
    messages = [
        {"role": "user", "content": prompt}
    ]
    
    data = {
        "model": llm,
        "messages": messages,
        "stream": False,
    }
    
    data = json.dumps(data)
    response = requests.post(uri, headers=headers, data=data).json()
    return response['message']['content']


In [6]:
r = oneshot("Write an SPL to search _internal index and splunkd sourcetype for the last 24 hours with keyword error pipe", 'mistral')
print(r)

 To create a Splunk Processing Language (SPL) query that searches for the keyword "error" in the internal index and from the sourcetype within the last 24 hours, you can use the following search pattern:

```
index=internal sourcetype=* error
| stats latest(_time) as last_search_time by _raw, sourcetype, index
| where _time > now(-1d)
| table sourcetype, index, last_search_time, _raw
```

Here's a breakdown of the query:

1. `index=internal` : Specify the index to search (in this case, the internal index).
2. `sourcetype=*` : Look for any sourcetype in this search. You can replace the asterisk (*) with the specific sourcetype if you want to focus on a particular one.
3. `error`: Filter events that contain the keyword "error".
4. `| stats latest(_time) as last_search_time by _raw, sourcetype, index` : Compute the most recent time for each event and group them by raw event, sourcetype, and index. The result is a table with columns: sourcetype, index, last_search_time (the latest time of 

In [7]:
r = oneshot("Write an SPL to search _internal index and splunkd sourcetype for the last 24 hours with keyword error pipe", 'llama3')
print(r)

Here is an example SPL command that searches the _internal index and the splunkd sourcetype for events containing the keyword "error" in the pipe field, within the last 24 hours:
```
index=_internal 
sourcetype=splunkd 
| where search match("pipe error") 
| timechart span=1h start=-24h
```
Let me explain what each part of this SPL command does:

* `index=_internal`: This specifies that we want to search the _internal index, which contains internal Splunk logs and metrics.
* `sourcetype=splunkd`: This specifies that we only want to look at events with the sourcetype "splunkd", which is the sourcetype for Splunk's own internal logs.
* `| where search match("pipe error")`: This part of the SPL uses the `where` command to filter the results. The `search` command searches for matches to the specified string ("pipe error" in this case), and the `match` function checks if that string is present in the pipe field of each event.
* `| timechart span=1h start=-24h`: This part of the SPL uses the 

## Stage 4 - apply the model

In [ ]:
# from fit command, we will pass parameters model and prompt.
# sample prompt: You will examine if the email content given by the user is phishing. 
#                Only output **Phishing** if the content is phishing. 
#                Only output **Legit** if the email is legitimate. Do not give extra information.
def apply(model,df,param):
    X = df["text"].values.tolist()
    uri = f"{ollama_url}/api/chat"
    headers = {'Content-Type': 'application/json'}
    outputs_label = []
    outputs_duration = []
    for i in range(len(X)):
        messages = [
            {"role": "user", "content": param['options']['params']['prompt'].strip("\"")},
            {"role": "user", "content": X[i]}
        ]
        
        data = {
            "model": param['options']['params']['model'].strip("\""),
            "messages": messages,
            "stream": False,
        }
        
        data = json.dumps(data)
        try:
            response = requests.post(uri, headers=headers, data=data).json()
            outputs_label.append(response['message']['content'])
            duration = round(int(response['total_duration']) / 1000000000, 2)
            duration = str(duration) + " s"
            outputs_duration.append(duration)
        except:
            outputs_label.append("ERROR")
            outputs_duration.append("ERROR")
        
    cols={'Result': outputs_label, 'Duration': outputs_duration}
    returns=pd.DataFrame(data=cols)
    return returns

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code